In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from src.misc import cumdiff

In [258]:
to_encode = list(cumdiff([10, 100, 123, 144, 3123, 3423, 10000, 14050, 100000]))

In [259]:
import math

In [260]:
[(bin(c)[2:], len(bin(c)[2:])) for c in to_encode]

[('1010', 4),
 ('1011010', 7),
 ('10111', 5),
 ('10101', 5),
 ('101110100011', 12),
 ('100101100', 9),
 ('1100110110001', 13),
 ('111111010010', 12),
 ('10100111110111110', 17)]

In [261]:
simple9 = [
    (28, 1),
    (14, 2),
    (9, 3),
    (7, 4),
    (5, 5),
    (4, 7),
    (3, 9),
    (2, 14),
    (1, 28),
]

In [262]:
bin(9)

'0b1001'

In [263]:
def simple9_encode(scheme, arr):
    _, bits = simple9[scheme]
    s = format(scheme, '04b')
    for a in arr:
        s += format(a, '0%ib' % bits)
    return struct.pack('I', int(s, 2))
    

def encode_partial(arr):
    count = 0
    is_fitted = False
    
    for scheme, (amount, bits) in enumerate(simple9):
        k = amount
        for c in arr:
            if len(bin(c)[2:]) <= bits:
                k -= 1
            else:
                break
            count += 1
            if k == 0:
                is_fitted = True
                break
        if is_fitted:
            return simple9_encode(scheme, arr[:count]), arr[count:]
        else:
            count = 0
            
def encode(arr):
    while True:
        code, arr = encode_partial(arr)
        yield code
        if len(arr) == 0:
            break

In [264]:
import textwrap

In [265]:
def decode(codes):
    for c in codes:
        c = format(struct.unpack('I', c)[0], '032b')
        amount, bits = simple9[int(c[:4], 2)]
        parts = textwrap.wrap(c[4:], bits)
        assert amount == len(parts)
        for p in parts:
            yield int(p, 2)

In [267]:
to_encode

[10, 90, 23, 21, 2979, 300, 6577, 4050, 85950]

In [268]:
list(encode(to_encode))

['\x95\x8bVQ', ',\xc1\xe8r', '\xd2Olv', '\xbeO\x01\x80']

In [269]:
list(decode(list(encode(to_encode))))

[10, 90, 23, 21, 2979, 300, 6577, 4050, 85950]

In [86]:
encode_partial(to_encode)

(4, 7, [3122977, 1000000])

In [ ]:
def encode(arr):
    count = 0
    is_fitted = False
    
    while(1):
        for amount, bits in simple9:
            for c in to_encode:
                if len(bin(c)[2:]) <= bits:
                    amount -= 1
                else:
                    break

                count += 1

                if amount == 0:
                    is_fitted = True
                    break

            if is_fitted:
                yield count

            else:
                count = 0

    

In [78]:
count = 0
is_fitted = False

for amount, bits in simple9:
    for c in to_encode:
        if len(bin(c)[2:]) <= bits:
            amount -= 1
        else:
            break
            
        count += 1
        
        if amount == 0:
            is_fitted = True
            break
            
    if is_fitted:
        print "GOOD"
    else:
        count = 0


7
6
4
3
22
20


In [43]:
len(bin(10)[2:])

4

In [257]:
import random
import struct

bits = "".join([str(random.randint(0, 1)) for _ in range(32)])
print bits
print len(bits)
print int(bits, 2)
packed = struct.pack('I', int(bits, 2))
print bits
print format(struct.unpack('I', packed)[0], '032b')

10111101000100011110000101010110
32
3172065622
10111101000100011110000101010110
10111101000100011110000101010110


In [181]:
# VarByte testing

In [308]:
from src.encoders import VarByte, Simple9

from src.generators import gen_cumdiff, gen_cumsum

In [309]:
arr = [10, 100, 123, 144, 3123, 3423, 10000, 14050, 100000]

In [310]:
vb = VarByte()
assert arr == list(gen_cumsum(vb.decode(vb.encode(gen_cumdiff(arr)))))

In [311]:
list(s9.encode(gen_cumdiff(arr)))

['\x95',
 '\x8b',
 'V',
 'Q',
 ',',
 '\xc1',
 '\xe8',
 'r',
 '\xd2',
 'O',
 'l',
 'v',
 '\xbe',
 'O',
 '\x01',
 '\x80']

In [312]:
s9 = Simple9()
assert arr == list(gen_cumsum(s9.decode(s9.encode(gen_cumdiff(arr)))))

In [294]:
import itertools

In [300]:
gen = (i for i in range(12))

In [306]:
list(itertools.islice(gen, 0, 4))

[]